In [159]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [161]:
# Set Working Directory
import os
working_dir = "/Users/muhammadsaad/Desktop/Data Science Project"
os.chdir(working_dir)

#### Scraping Fox 5 Washington DC Page ####

In [163]:
# Function to scrape 3 pages at a time
def scrape_pages(start_page, total_pages=3):
    news_data = []
    for page in range(start_page, start_page + total_pages):
        url = f"https://www.fox5dc.com/tag/us/dc?page={page}"  # Use 'page' variable here
        print(f"Scraping page {page}: {url}")
        
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
        
        articles = soup.select(".post")  # Change this selector if needed

        for article in articles:
            title_tag = article.select_one(".title a")
            title = title_tag.text.strip() if title_tag else "No title"
            
            summary_tag = article.select_one(".dek")
            summary = summary_tag.text.strip() if summary_tag else "No summary"
            
            date_tag = article.select_one(".time")
            date = date_tag.text.strip() if date_tag else "No date"
            
            article_link = title_tag['href'] if title_tag else None
            full_article = get_full_article(article_link) if article_link else "No full article"
            
            news_data.append({
                "Title": title,
                "Summary": summary,
                "Date": date,
                "Full_Article": full_article
            })
        
        time.sleep(5)  # Sleep to avoid getting blocked
    
    # Convert the results to a pandas DataFrame
    news_df = pd.DataFrame(news_data)
    
    # Save to CSV
    news_df.to_csv(f"news_data_{start_page}-{start_page + total_pages - 1}.csv", index=False)
    
    # Display the DataFrame
    return news_df

In [165]:
# Function to fetch the full article (if necessary)
def get_full_article(url):
    if url is None:
        return "No full article"
    
    full_url = f"https://www.fox5dc.com{url}"  # Ensure the full URL is formed
    response = requests.get(full_url)
    soup = BeautifulSoup(response.content, "html.parser")
    
    # Extract the article content
    article_content = soup.select_one(".article-body p")  # Adjust the selector based on the actual structure
    
    if article_content:
        return article_content.get_text(strip=True)
    return "No content available"

In [173]:
# Scrape 3 pages at a time, starting from page 1
start_page = 10  # Change this to the page number you want to start from
news_df = scrape_pages(start_page, total_pages=3)

# Display the DataFrame in Jupyter
import IPython.display as display
display.display(news_df)

Scraping page 10: https://www.fox5dc.com/tag/us/dc?page=10


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))